In [326]:
import pandas as pd
import yfinance as yf

In [327]:
# df_stock = yf.Ticker('^GSPC').history(start='2004-01-1', interval='1mo').iloc[::12, :]
# df_bond = yf.Ticker('AGG').history(start='2004-01-01', interval='1mo').iloc[::12, :]
# df_comm = yf.Ticker('^SPGSCI').history(start='2004-01-01', interval='1mo').iloc[::12, :]
df_stock = yf.Ticker('AAPL').history(start='2015-01-01', interval='1mo').iloc[::12, :]
df_bond = yf.Ticker('MSFT').history(start='2015-01-01', interval='1mo').iloc[::12, :]
df_comm = yf.Ticker('TSLA').history(start='2015-01-01', interval='1mo').iloc[::12, :]
df_portfolio = pd.DataFrame({
    'Stock': df_stock.Close.pct_change(),
    'Bond': df_bond.Close.pct_change(), 
    'Commodity': df_comm.Close.pct_change()}, index=df_stock.index).dropna()
df_portfolio.describe()

,Stock,Bond,Commodity
count,9.000000,9.000000,9.000000
mean,0.287344,0.337017,0.729309
std,0.361400,0.265049,1.695106
min,-0.169657,-0.195665,-0.445234
25%,0.009011,0.205736,-0.060904
50%,0.285125,0.377062,0.180447
75%,0.401915,0.502420,0.406383
max,0.887530,0.654190,5.098729


$$
\frac{\partial}{\partial A} \left( \frac{A}{\sqrt{B - A^2}} \right) = \frac{B}{(B - A^2)^{3/2}}
$$
$$
\frac{\partial}{\partial B} \left( \frac{A}{\sqrt{B - A^2}} \right) = -\frac{A}{2 \sqrt{B - A^2}}
$$
$$
\frac{\partial}{\partial R_i} \frac{1}{T} \sum_{i=1}^{T} R_i^2 = \frac{1}{T} \sum_{i=1}^{T} \frac{\partial}{\partial R_i} R_i^2 = \frac{2}{T} \sum_{i=1}^{T} R_i = 2E[R]
$$

In [335]:
import numpy as np

w = np.array([1/3, 1/3, 1/3])
max_score = 0
max_w = []

for _ in range(10000):
    R = np.matmul(df_portfolio.to_numpy(), w)
    A = R.mean()
    B = np.power(R, 2).mean()
    S_denom = B-np.power(A, 2)
    dS_dA = B/np.power(S_denom, 3/2)
    dA_dw = (df_portfolio*w).mean().to_numpy()
    dS_dB = A/(-2*np.sqrt(S_denom))
    dB_dw = np.array([2*(df_portfolio.iloc[:, i].to_numpy()*R).mean() for i in range(df_portfolio.shape[1])])
    dB_dR = 2*A
    #grad = (dS_dA*dA_dw)+(dS_dB*dB_dw)
    grad = (dS_dA+dS_dB*dB_dR*R).sum()
    w += 0.0001*grad
    if any(v < 0 for v in w):
        break
    w /= w.sum()
    score = R.mean()/R.std()
    if score > max_score:
        max_w = w
        max_score = score
np.set_printoptions(suppress=True)
print(max_score, max_w)

0.6991263911586268 [0.33333333 0.33333333 0.33333333]


In [329]:
import numpy as np
from scipy.optimize import minimize

# Define the function to be optimized
def func(x):
    numerator = np.matmul(df_portfolio, x).mean()
    denominator = np.sqrt(np.power(np.matmul(df_portfolio, x), 2).mean() - numerator**2)
    return -numerator / denominator  # Negative because we're finding the maximum

# Initial guess
x0 = [1/3, 1/3, 1/3]

# Define bounds for x, y, and z (if needed)
bounds = [(0, 1), (0, 1), (0, 1)]

# Minimize the negative of the function to find the maximum
result = minimize(func, x0, bounds=bounds, constraints={'type': 'eq', 'fun': lambda x: sum(x)-1})

# The result object contains the maximum value and the corresponding values of x, y, and z
max_value = -result.fun
max_x, max_y, max_z = result.x

print("Maximum value:", max_value)
print("Values of x, y, z at maximum:", max_x, max_y, max_z, sum([max_x, max_y, max_z]))

Maximum value: 1.3551885985740708
Values of x, y, z at maximum: 0.0 0.9839258647704936 0.01607413522950679 1.0000000000000004
